In [1]:
import pandas as pd
import polars as pl
import numpy as np
import sqlalchemy
import os
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
from math import floor
from dateutil.relativedelta import relativedelta

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases


<h3>Definição dos parametros</h3>

In [2]:
data = datetime.today().strftime("%d-%m-%Y")
diretorio_base = "C:\\Scripts\\relatórios\\"
horario = os.listdir("{}{}".format(diretorio_base, data))[0]

<h1><center>Migração das tabelas</center></h1>

#####  Tabela times

In [3]:
def times_novo_table():
    times = pl.read_excel("C:\\Scripts\\times_atria.xlsx")
    times = times.with_columns(
        pl.col("Assessor").str.to_uppercase()
    )
    return times

In [4]:
times = times_novo_table()

print("--- Visualizando DataFrame (Polars) ---")
print(times.head(20)) # Mostra as 10 primeiras linhas
print(f"Shape: {times.shape}") # Mostra quantas linhas e colunas tem

conexao = Connect.connect_techdb()
times.to_pandas().to_sql("times_nova_empresa", 
                         con = conexao, 
                         index = False, 
                         if_exists = "replace",
                         schema="principal")
conexao.execute(text('ALTER TABLE principal."times_nova_empresa" ADD PRIMARY KEY ("Assessor")'))
conexao.close()

--- Visualizando DataFrame (Polars) ---
shape: (15, 3)
┌─────────────────────────────────┬────────────┬─────────────────────────────────┐
│ Assessor                        ┆ TIME       ┆ Email                           │
│ ---                             ┆ ---        ┆ ---                             │
│ str                             ┆ str        ┆ str                             │
╞═════════════════════════════════╪════════════╪═════════════════════════════════╡
│ FERNANDO DOMINGUES DA SILVA     ┆ SÃO JOSÉ   ┆ fernando.domingues@atriacm.com… │
│ PAULO ROBERTO FARIA SILVA       ┆ SÃO JOSÉ   ┆ paulo.faria@atriacm.com.br      │
│ SAADALLAH JOSE ASSAD            ┆ SÃO JOSÉ   ┆ saadallah.assad@atriacm.com.br  │
│ RODRIGO DE MELLO D’ELIA         ┆ PIRACICABA ┆ rodrigo.delia@atriacm.com.br    │
│ ROSANA APARECIDA PAVANI DA SIL… ┆ SÃO PAULO  ┆ rosana.pavani@atriacm.com.br    │
│ …                               ┆ …          ┆ …                               │
│ MARCOS SOARES PEREIRA FILHO   

##### Tipo clientes

In [5]:
conexao = Connect.connect_techdb()
base_btg = Connect.import_table(conexao, "base_btg")
tipo_clientes_historico = Connect.import_table(conexao, "tipo_clientes")
conexao.close()

In [6]:
tipo_clientes = base_btg.loc[:,["Conta", "Tipo"]]
tipo_clientes = tipo_clientes[~tipo_clientes['Conta'].isin(tipo_clientes_historico['Conta'])]
tipo_clientes.loc[tipo_clientes['Tipo'].isnull(), "Tipo"] = 'Offshore'

In [7]:
conexao = Connect.connect_techdb()
tipo_clientes.to_sql("tipo_clientes", 
                     con=conexao, 
                     index=False, 
                     if_exists='append',
                     schema="principal")
try:
    conexao.execute(str('ALTER TABLE principal."tipo_clientes" ADD PRIMARY KEY ("Conta")'))
except:
    pass
conexao.close()

##### Base BTG

In [8]:
base = pd.read_excel(diretorio_base+"%s\\%s\\base_btg.xlsx" % (data, horario), header=2)
base.rename(columns={"E-Mail Comunicação":"E-mail"}, inplace=True)

for columns in base.columns:
    base.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
base['Conta'] = base['Conta'].astype(str)

In [9]:
base['Assessor'] = base['Assessor'].apply(lambda nome: str(nome).upper())
base.loc[base['Faixa Cliente'] == "Ate 50K", "Faixa Cliente"] = "Até 300k"
base.loc[base['Faixa Cliente'] == "Entre 50k e 100k", "Faixa Cliente"] = "Até 300k"
base.loc[base['Faixa Cliente'] == "Entre 100k e 300k", "Faixa Cliente"] = "Até 300k"

In [10]:
offshore = pd.read_excel(r"C:\Scripts\historico_auc\AuC Offshore.xlsx", sheet_name='AuC Offshore')
offshore = offshore.loc[:,["Nome", "Conta", "AUC BRL", "Assessor"]]
offshore.rename(columns={"AUC BRL":"PL Total"}, inplace=True)
offshore['Conta'] = offshore['Conta'].astype(str)
offshore['Assessor'] = [nome.upper() for nome in offshore['Assessor']]

In [11]:
print(offshore.head(20)) # Mostra as 20 primeiras linhas

                       Nome       Conta      PL Total  \
0                   Tamires  BPAB000028  4.473687e+04   
1   NDFJ PARTICIPACOES LTDA    BP000000  1.982880e+06   
2                     Kelli  BPRG000135  1.128163e+05   
3                 Alexandre  BPJP000126  9.835424e+04   
4                  Henrique  BPCH000006  6.376478e+04   
5                   Evandro  BPPE000110  1.747270e+03   
6                    Romulo  BPEN000130  8.150379e+05   
7                    Eliana  BPCU000144  3.729337e+04   
8                      Jana  BPJN000143  1.984285e+04   
9                    Franco  BPHF000138  1.656435e+04   
10                  Camilla  BPXS000122  9.999595e+04   
11                   Flavio  BPEP000040  1.478576e+05   
12                   Clovis  BPMC000038  1.134777e+04   
13                    Ana    BPUX000073  5.843276e+05   
14                   Homero  BPTC000147  5.793296e+05   
15                    Breno  BPYM000138  1.130976e+05   
16                  Miralda  BP

In [12]:

conexao = Connect.connect_techdb()
offshore.to_sql(
    "pl_offshore",
    con=conexao,
    index=False,
    if_exists="replace",
    dtype={"Conta": sqlalchemy.VARCHAR(255)},
    schema="principal"
)

conexao.execute(
    text('ALTER TABLE principal."pl_offshore" ADD PRIMARY KEY ("Conta")')
)

conexao.close()

In [13]:
base = pd.concat([offshore, base], axis=0)

In [14]:
#### Ajuste assessores reais
base.loc[base['Assessor'] == "MURILO LUIZ SILVA GINO", "Assessor"] = "IZADORA VILLELA FREITAS"
base.loc[base['Assessor'].str.contains("GABRIEL GUERRERO TORRES FONSECA"), "Assessor"] = "MARCOS SOARES PEREIRA FILHO"

In [15]:
base.drop_duplicates(subset="Conta", keep='first', inplace=True)
base.rename(columns={"Perfil de Cliente":"Perfil do Cliente"}, inplace=True)

In [16]:
### Ajuste nome do rodrigo

base.loc[base['Assessor'] == "Rodrigo de Mello D?Elia", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
base.loc[base['Assessor'] == "Rodrigo de Mello DElia", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
base.loc[base['Assessor'] == "RODRIGO DE MELLO DELIA", "Assessor"] = "RODRIGO DE MELLO D’ELIA"

In [17]:
conexao = Connect.connect_techdb()
base.to_sql("base_btg",
            con=conexao, 
            index=False, 
            if_exists="replace", 
            schema="principal",
            dtype={"Conta":sqlalchemy.VARCHAR(255)})
conexao.execute(text('ALTER TABLE principal."base_btg" ADD PRIMARY KEY ("Conta")'))
conexao.close()

#### Acompanhamento de custódia


In [18]:
files_diretorio_base = os.listdir(diretorio_base+"%s" % (data))
files_diretorio_base = [element for element in files_diretorio_base if 'acompanhamento-de-custodia' in element]
files_diretorio_base = files_diretorio_base[0]

acompanhamento_de_custodia = pd.read_excel(diretorio_base+"%s\\%s" % (data, files_diretorio_base))

c:\Scripts\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
conexao = Connect.connect_techdb()
acompanhamento_de_custodia.to_sql('acompanhamento_custodia', 
                                    con=conexao, 
                                    index=False, 
                                    if_exists='replace',
                                    schema="principal")
conexao.close()

##### Nomes completos

In [20]:
nomes_clientes = pd.read_excel(r"C:\Scripts\nomes_clientes\Nomes_Clientes2.xlsx")
nomes_clientes.rename(columns={"Código":"Conta"}, inplace=True)
nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str)
nomes_clientes = nomes_clientes[~nomes_clientes['Nome'].isna()]

In [21]:
base_btg = base[['Conta']]
nomes_clientes = pd.merge(base_btg, nomes_clientes, on="Conta", how='outer')

In [22]:
for conta_sem_nome in nomes_clientes[nomes_clientes['Nome'].isnull()]['Conta']:
    try:
        nomes_clientes.loc[nomes_clientes['Conta'] == conta_sem_nome, 'Nome'] = \
            base[base['Conta'] == conta_sem_nome]['Nome'].values[0]
    except:
        pass

In [23]:
nomes_clientes['Conta'] = nomes_clientes['Conta'].astype(str)
nomes_clientes.drop_duplicates(subset='Conta', inplace=True)

In [24]:
nomes_clientes.to_excel(r"C:\Scripts\nomes_clientes\Nomes_Clientes2.xlsx", header=True, index=False)
conexao = Connect.connect_techdb()
nomes_clientes.to_sql('nomes_clientes', 
                      con=conexao, 
                      index=False, 
                      if_exists='replace',
                      schema="principal")
conexao.execute(text('ALTER TABLE principal."nomes_clientes" ADD PRIMARY KEY ("Conta")'))
conexao.close()

<h3>Saldo CC</h3>

In [25]:
saldo_cc = pd.read_excel(diretorio_base+"%s\\%s\\saldo_cc.xlsx" % (data, horario), header=2)
for columns in saldo_cc.columns:
    saldo_cc.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
saldo_cc.rename(columns={"Saldo":"SALDO"}, inplace=True)
saldo_cc = saldo_cc.loc[:,["Conta", "SALDO"]]
saldo_cc['Conta'] = saldo_cc['Conta'].astype(str)
saldo_cc = pd.merge(saldo_cc, base[['Assessor', 'Conta']], on='Conta', how='left')

conexao = Connect.connect_techdb()
saldo_cc.to_sql("saldo_conta_corrente",
                con=conexao, 
                index=False, 
                if_exists='replace', 
                dtype={"Conta":sqlalchemy.VARCHAR(255)},
                schema="principal")
conexao.execute(text('ALTER TABLE principal."saldo_conta_corrente" ADD PRIMARY KEY ("Conta")'))
conexao.close()

<h2>Posição</h2>

In [26]:
posicao = pd.read_excel(diretorio_base+"%s\\%s\posição.xlsx" % (data, horario), header=2)
for columns in posicao.columns:
    posicao.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
posicao.rename(columns={'CONTA':'Conta', 'Submercado':'Sub Mercado'}, inplace=True)
posicao['Conta'] = posicao['Conta'].astype(str)
posicao.rename(columns={"Vencimento":"VENCIMENTO", "Escritório":"ESCRITÓRIO"}, inplace=True)
posicao['VENCIMENTO'] = pd.to_datetime(posicao['VENCIMENTO'], format="%Y-%m-%d", errors='coerce')
posicao = pd.merge(posicao, base[['Conta','Assessor']], on='Conta', how='left')
posicao.rename(columns={"IR":"Soma de IR", "IOF":"Soma de IOF"}, inplace=True)
posicao.drop("ESCRITÓRIO", axis=1, inplace=True)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\joaob\AppData\Local\Temp\ipykernel_25956\2432758331.py:1: SyntaxWarning: invalid escape sequence '\p'
  posicao = pd.read_excel(diretorio_base+"%s\\%s\posição.xlsx" % (data, horario), header=2)


In [27]:
setores = pd.read_excel(r"C:\Scripts\setores_ativos\setores.xlsx")

In [28]:
posicao['Setor'] = ''
posicao['Subsetor'] = ''
for ativo, emissor in zip(setores['Ativo'], setores['Emissor']):
    try:
        posicao.loc[(posicao['Ativo'] == ativo) | (posicao['Emissor'] == emissor), 'Setor'] = setores[setores['Ativo'] == ativo]['SETOR_ECONÔMICO'].values[0]
        posicao.loc[(posicao['Ativo'] == ativo) | (posicao['Emissor'] == emissor), 'Subsetor'] = setores[setores['Ativo'] == ativo]['SUBSETOR'].values[0]
    except:
        pass

In [29]:
conexao = Connect.connect_techdb()
posicao.to_sql('posicao', 
               index=False, 
               con=conexao, 
               if_exists='replace', 
               dtype={'CONTA':sqlalchemy.VARCHAR(255)},
               schema="principal")
conexao.close()

<h2>Renda fixa</h2>

In [30]:
renda_fixa = pd.read_excel(diretorio_base+"%s\\%s\\renda_fixa_coe.xlsx" % (data, horario), header=2)

In [31]:
for columns in renda_fixa.columns:
    renda_fixa.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)

In [32]:
renda_fixa = pd.read_excel(diretorio_base+"%s\\%s\\renda_fixa_coe.xlsx" % (data, horario), header=2)
renda_fixa.rename(columns={"'DL_D_ContaAssessor'[NR_CONTA]":"Conta"}, inplace=True)
renda_fixa['Conta'] = renda_fixa['Conta'].astype(str)
renda_fixa.drop("ID", axis=1, inplace=True)

In [33]:
conexao = Connect.connect_techdb()
renda_fixa.to_sql('renda_fixa', 
                  index=False, 
                  con=conexao, 
                  if_exists='replace', 
                  dtype={"CONTA":sqlalchemy.VARCHAR(255)},
                  schema="principal")
conexao.close()

<h2>Renda varíavel</h2>

In [34]:
renda_variavel = pd.read_excel(diretorio_base+"%s\\%s\\renda_variavel.xlsx" % (data, horario), header=2)

In [35]:
for columns in renda_variavel.columns:
    renda_variavel.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)

In [36]:
renda_variavel = pd.read_excel(diretorio_base+"%s\\%s\\renda_variavel.xlsx" % (data, horario), header=2)
renda_variavel.rename(columns={"Data Vencimento":"DATA VENCIMENTO"}, inplace=True)
renda_variavel['DATA VENCIMENTO'] = pd.to_datetime(renda_variavel['DATA VENCIMENTO'], format=("%Y-%m-%d"), errors='coerce')
renda_variavel.rename(columns={"'DL_D_ContaAssessor'[NR_CONTA]":"Conta"}, inplace=True)
renda_variavel['Conta'] = renda_variavel['Conta'].astype(str)
renda_variavel['Valor Bruto Inicial'] = renda_variavel['Preço Médio']*renda_variavel['Quantidade']
renda_variavel['Performance'] = ((renda_variavel['Valor Bruto']-renda_variavel['Valor Bruto Inicial'])/renda_variavel['Valor Bruto Inicial'])*100
renda_variavel.loc[renda_variavel['Performance'] == np.inf, "Performance"] = 0

In [37]:
conexao = Connect.connect_techdb()
renda_variavel.to_sql('renda_variavel', 
                      index=False, 
                      con=conexao, 
                      if_exists='replace', 
                      dtype={"CONTA":sqlalchemy.VARCHAR(255)},
                      schema="principal")
conexao.close()

<h2>Rentabilidade</h2>

In [38]:
rentabilidade = pd.read_excel(diretorio_base+"%s\\%s\\rentabilidade.xlsx" % (data, horario), header=2)
rentabilidade['Conta'] = rentabilidade['Conta'].astype(str)

In [39]:
rentabilidade = \
    rentabilidade.loc[:,["Conta", "Data", 'Profitability mtd',
        'Profitability last 12 months accumulated',
        'Profitability last 6 months accumulated',
        'Profitability last 3 months accumulated', 'Profitability ytd',
        'Modelo de Rentabilidade']]

In [40]:
conexao = Connect.connect_techdb()
rentabilidade.to_sql('rentabilidade', 
                        index=False, 
                        con=conexao, 
                        if_exists='replace', 
                        dtype={"Conta":sqlalchemy.VARCHAR(255)},
                        schema="principal")
conexao.close()

#### PL Base

In [41]:
# Hoje
base_hoje = pd.read_excel(diretorio_base+"%s\\%s\\base_btg.xlsx" % (data, horario), header=2)
for columns in base_hoje.columns:
    base_hoje.rename(columns={columns:columns.replace(" (R$)","")}, inplace=True)
base_hoje['Conta'] = base_hoje['Conta'].astype(str)

base.rename(columns={'CONTA':'Conta'}, inplace=True)
base['Conta'] = base['Conta'].astype(str)

In [42]:
conexao = Connect.connect_techdb()
pl_base_hist = Connect.import_table(conexao, "PL Base")
conexao.close()

In [43]:
def tratar_base(base):
    base = base[['Assessor', 'Conta', 'PL Total']]
    base.rename(columns={'PL Total':'PL'}, inplace=True)
    return base

In [44]:
base_hoje = tratar_base(base_hoje)
base_hoje['Mês'] = datetime.today().strftime("%Y-%m-%d")

C:\Users\joaob\AppData\Local\Temp\ipykernel_25956\2034323585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base.rename(columns={'PL Total':'PL'}, inplace=True)


In [45]:
pl_base_hist = pl_base_hist[pl_base_hist['Mês'] < datetime.today().strftime("%Y-%m-01")]

In [46]:
base_hoje.rename(columns={"Conta":"CONTA"}, inplace=True)

In [47]:
## Concatenar todos os meses
pl_mes_vigente = pd.concat([pl_base_hist,
                            base_hoje], axis=0)

pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "Murilo Luiz Silva Gino", "Assessor"] = "Fernando Domingues da Silva"

In [48]:
conexao = Connect.connect_techdb()
pl_offshore_hist = Connect.import_table(conexao, "offshore_adicionar_pl_mes_vigente")
offshore_pl = Connect.import_table(conexao, "pl_offshore")
conexao.close()

In [49]:
offshore_pl

,Nome,Conta,PL Total,Assessor
0,Tamires,BPAB000028,4.473687e+04,GIORDANO LONGONI LARA
1,NDFJ PARTICIPACOES LTDA,BP000000,1.982880e+06,PAULO ROBERTO FARIA SILVA
2,Kelli,BPRG000135,1.128163e+05,RAFAEL PASOLD MEDEIROS
3,Alexandre,BPJP000126,9.835424e+04,FERNANDO DOMINGUES DA SILVA
4,Henrique,BPCH000006,6.376478e+04,GIORDANO LONGONI LARA
5,Evandro,BPPE000110,1.747270e+03,PAULO ROBERTO FARIA SILVA
6,Romulo,BPEN000130,8.150379e+05,RAFAEL PASOLD MEDEIROS
7,Eliana,BPCU000144,3.729337e+04,RODRIGO DE MELLO DELIA
8,Jana,BPJN000143,1.984285e+04,FELIPE AUGUSTO CARDOSO
9,Franco,BPHF000138,1.656435e+04,GIORDANO LONGONI LARA


In [50]:
pl_offshore_hist = pl_offshore_hist[pl_offshore_hist['Mês'] < datetime.today().strftime("%Y-%m-01")]
pl_offshore_hist['Mês'] = [pd.to_datetime(data).strftime("%Y-%m-%d") for data in pl_offshore_hist['Mês']]
offshore_pl['Mês'] = datetime.today().strftime("%Y-%m-%d")
offshore_pl.rename(columns={"PL Total":"PL", "Conta":"CONTA"}, inplace=True)
offshore_pl.drop("Nome", axis=1, inplace=True)

In [51]:
adicionar_offshore_pl = pd.concat([offshore_pl, pl_offshore_hist], axis=0)

In [52]:
conexao = Connect.connect_techdb()
adicionar_offshore_pl.to_sql("offshore_adicionar_pl_mes_vigente", 
                            index=False, 
                            con=conexao, 
                            if_exists='replace',
                            schema="principal")
conexao.close()

In [53]:
#contas = base['Conta']
#pl_mes_vigente = pl_mes_vigente[pl_mes_vigente['Conta'].isin(contas)]
pl_mes_vigente['PL'] = pl_mes_vigente['PL'].fillna(0)
pl_mes_vigente['Assessor'] = pl_mes_vigente['Assessor'].apply(lambda nome: str(nome).upper())
pl_mes_vigente = pd.concat([pl_mes_vigente, adicionar_offshore_pl], axis=0)
pl_mes_vigente['Mês'] = [pd.to_datetime(data) for data in pl_mes_vigente['Mês']]

In [54]:
pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "RODRIGO DE MELLO DELIA", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "ROSANA PAVANI", "Assessor"] = "ROSANA APARECIDA PAVANI DA SILVA"
pl_mes_vigente.loc[pl_mes_vigente['Assessor'] == "FERNANDO DOMINGUES", "Assessor"] = "FERNANDO DOMINGUES DA SILVA"

In [55]:
pl_mes_vigente['CONTA'] = pl_mes_vigente['CONTA'].astype(str)

In [56]:
pl_mes_vigente = pl_mes_vigente.drop_duplicates(subset=["CONTA", "Mês"], keep='first')

In [57]:
conexao = Connect.connect_techdb()
pl_mes_vigente.to_sql('PL Base', 
                      index=False, 
                      con=conexao, 
                      if_exists='replace',
                      schema="principal")
conexao.close()

In [58]:
print("Ultima atualização: ", datetime.today())
import winsound
frequency = 2500  # fequência em hertz
duration = 500  # duração. 500 = 0,5s
winsound.Beep(frequency, duration)

Ultima atualização:  2026-02-05 09:11:32.577344


In [59]:
from sqlalchemy import column

horario = [datetime.today().strftime("%d/%m/%Y %H:%M:%S")]
horario_atualizacao = pd.DataFrame(horario, columns=["Horario"])

In [60]:
horario_atualizacao

,Horario
0,05/02/2026 09:11:33


In [61]:
conexao = Connect.connect_techdb()
horario_atualizacao.to_sql(
    "horario_atualizacao_relatorios", 
    index=False, 
    con=conexao, 
    if_exists='replace', 
    schema="principal")
conexao.close()